In [2]:
import numpy as np   # Downloading numpy for array format
import pandas as pd  # Downloading pandas for data import
import matplotlib.pyplot as plt # Matplotlib.pyplot for visualization
%matplotlib inline 

In [3]:
oil_prices=pd.read_csv('crude_oil_import_prices.csv') # importing the dataset

oil_prices.head(10)

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,OILIMPPRICE,TOT,USD_BAR,A,1980,31.81,NaN
1,AUS,OILIMPPRICE,TOT,USD_BAR,A,1981,35.88,NaN
2,AUS,OILIMPPRICE,TOT,USD_BAR,A,1982,35.42,NaN
3,AUS,OILIMPPRICE,TOT,USD_BAR,A,1983,30.88,NaN
4,AUS,OILIMPPRICE,TOT,USD_BAR,A,1984,29.19,NaN
5,AUS,OILIMPPRICE,TOT,USD_BAR,A,1985,28.17,NaN
6,AUS,OILIMPPRICE,TOT,USD_BAR,A,1986,14.49,NaN
7,AUS,OILIMPPRICE,TOT,USD_BAR,A,1987,19.00,NaN
8,AUS,OILIMPPRICE,TOT,USD_BAR,A,1988,15.93,NaN
9,AUS,OILIMPPRICE,TOT,USD_BAR,A,1989,17.63,NaN


In [6]:
oil_prices.shape # Getting shape  of data

(17360, 8)

In [9]:
oil_prices.info()   # checking nulls and data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17360 entries, 0 to 17359
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCATION    17360 non-null  object 
 1   INDICATOR   17360 non-null  object 
 2   SUBJECT     17360 non-null  object 
 3   MEASURE     17360 non-null  object 
 4   FREQUENCY   17360 non-null  object 
 5   TIME        17360 non-null  object 
 6   Value       9734 non-null   float64
 7   Flag Codes  7626 non-null   object 
dtypes: float64(1), object(7)
memory usage: 1.1+ MB


In [10]:
oil_prices.dtypes

LOCATION       object
INDICATOR      object
SUBJECT        object
MEASURE        object
FREQUENCY      object
TIME           object
Value         float64
Flag Codes     object
dtype: object

In [15]:
oil_prices=oil_prices.drop(['INDICATOR', 'SUBJECT', 'MEASURE', 'Flag Codes'], axis=1)

oil_prices.head(5)

,LOCATION,FREQUENCY,TIME,Value
0,AUS,A,1980,31.81
1,AUS,A,1981,35.88
2,AUS,A,1982,35.42
3,AUS,A,1983,30.88
4,AUS,A,1984,29.19


In [16]:
oil_prices.dtypes

LOCATION      object
FREQUENCY     object
TIME          object
Value        float64
dtype: object

In [17]:
oil_prices.shape

(17360, 4)

In [20]:
oil_prices.describe()     # Basic statistical info - seems to skew to the left

,Value
count,9734.000000
mean,41.376256
std,31.575795
min,2.330000
25%,18.350000
50%,28.560000
75%,56.265000
max,142.290000


In [23]:
oil_prices=oil_prices.sort_values(by='TIME') # sort the data by year

oil_prices.head(10)
oil_prices.tail(5)

,LOCATION,FREQUENCY,TIME,Value
4293,CHE,Q,2016-Q2,NaN
1519,CAN,Q,2016-Q2,44.27
1665,CZE,Q,2016-Q2,NaN
3125,NLD,Q,2016-Q2,43.29
3709,PRT,Q,2016-Q2,NaN


In [27]:
oil_prices['TIME']=oil_prices['TIME'].str.slice(0, 4) # Removed extra characters from time column

oil_prices.tail(5)


,LOCATION,FREQUENCY,TIME,Value
4293,CHE,Q,2016,NaN
1519,CAN,Q,2016,44.27
1665,CZE,Q,2016,NaN
3125,NLD,Q,2016,43.29
3709,PRT,Q,2016,NaN
